In [26]:
# LOAD PTB-XL SCRIPT AS PER LITERATURE PAPER


In [15]:
# Step 1 - Import libraries
# -------------------------------------------------------------------#
import pandas as pd
import numpy as np
import wfdb
import ast
import pandas as pd
import os
import matplotlib.pyplot as plt
import math
import numpy as np
import statistics
from keras.layers import Input, Dense, Dropout
from keras.layers import Dense
from keras.models import Model, load_model
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "browser"


In [16]:
# Step 2 - Define function to load raw data
# -------------------------------------------------------------------
def load_raw_data(df, sampling_rate, path):
    if sampling_rate == 100:
        data = [wfdb.rdsamp(path + f) for f in df.filename_lr]
    else:
        data = [wfdb.rdsamp(path + f) for f in df.filename_hr]
    data = np.array([signal for signal, meta in data])
    return data


In [17]:
# Define path and sampling rate at which to load data
# -------------------------------------------------------------------
path = '/Users/sylviachadha/Desktop/Thesis/Datasets/ptb-xl/'
sampling_rate = 100


In [18]:
# Step 3 - Read whole database file which has 27 col's including scp_codes
# Load and convert annotation data
# ------------------------------------------------------------------------
Y = pd.read_csv(path + 'ptbxl_database.csv', index_col='ecg_id')
Y.scp_codes = Y.scp_codes.apply(lambda x: ast.literal_eval(x))
# ast.literal_eval raises an exception if the input isn't a valid Python
# datatype, so the code won't be executed if it's not
print(Y.columns)


Index(['patient_id', 'age', 'sex', 'height', 'weight', 'nurse', 'site',
       'device', 'recording_date', 'report', 'scp_codes', 'heart_axis',
       'infarction_stadium1', 'infarction_stadium2', 'validated_by',
       'second_opinion', 'initial_autogenerated_report', 'validated_by_human',
       'baseline_drift', 'static_noise', 'burst_noise', 'electrodes_problems',
       'extra_beats', 'pacemaker', 'strat_fold', 'filename_lr', 'filename_hr'],
      dtype='object')


In [19]:
# Step 4 - Load raw signal data
# Reads all files, Y is whole db with 27 columns
# -------------------------------------------------------------------

X = load_raw_data(Y, sampling_rate, path)


In [20]:
# Input Shape
print(X.shape)


(21837, 1000, 12)


In [21]:
# Step 5 - Load scp_statements.csv for diagnostic aggregation
# -------------------------------------------------------------------

agg_df = pd.read_csv(path + 'scp_statements.csv', index_col=0)
# Out of form, rhythm and diagnostic they are separating diagnostic only
agg_df = agg_df[agg_df.diagnostic == 1]

In [22]:
# Step 6 - Adding superclass and subclass to original database
# -------------------------------------------------------------------

def aggregate_diagnostic(y_dic):
    tmp = []
    for key in y_dic.keys():
        if key in agg_df.index:
            tmp.append(agg_df.loc[key].diagnostic_class)
    return list(set(tmp))


# Apply diagnostic superclass
Y['diagnostic_superclass'] = Y.scp_codes.apply(aggregate_diagnostic)

In [23]:
def aggregate_diagnostic1(y_dic):
    tmp = []
    for key in y_dic.keys():
        if key in agg_df.index:
            tmp.append(agg_df.loc[key].diagnostic_subclass)
    return list(set(tmp))


# Apply diagnostic subclass
Y['diagnostic_subclass'] = Y.scp_codes.apply(aggregate_diagnostic1)

In [24]:
# -------------------------------------------#
# PART C - Step 10 - Train-Test Split
# -------------------------------------------#

# Split data into train and test ### Original
# 10 fold stratified sampling
test_fold = 10
# Train
X_train = X[np.where(Y.strat_fold != test_fold)]
y_train = Y[(Y.strat_fold != test_fold)].diagnostic_subclass
# Test
X_test = X[np.where(Y.strat_fold == test_fold)]
y_test = Y[Y.strat_fold == test_fold].diagnostic_subclass


In [25]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

print(y_train.head())
print(y_test.head())


(19634, 1000, 12)
(19634,)
(2203, 1000, 12)
(2203,)
ecg_id
1    [NORM]
2    [NORM]
3    [NORM]
4    [NORM]
5    [NORM]
Name: diagnostic_subclass, dtype: object
ecg_id
9     [NORM]
38    [NORM]
40    [NORM]
57    [NORM]
59    [NORM]
Name: diagnostic_subclass, dtype: object
